# First Part
### Data and Features
The dataset H_MHAS_c2.sas7bdat has 26839 rows and 5241 features.

SOME IMPORTANT INSIGHTS
-   The first character of the majority of variables indicates whether the variable refers to the reference person (“r”), spouse (“s”), or household (“h”).
-   The second character indicates the wave to which the variable pertains: “1”, “2”, “3”, “4”, “5”, or “A”. The “A” indicates “all,”

All features are divided into the following sections.

- SECTION A: DEMOGRAPHICS, IDENTIFIERS, AND WEIGHTS
- SECTION B: HEALTH
- SECTION C: HEALTH CARE UTILIZATION AND INSURANCE
- SECTION D: COGNITION
- SECTION E: FINANCIAL AND HOUSING WEALTH
- SECTION F: INCOME
- SECTION G: FAMILY STRUCTURE
- SECTION H: EMPLOYMENT HISTORY
- SECTION I: RETIREMENT
- SECTION J: PENSION
- SECTION K: PHYSICAL MEASURES
- SECTION L: ASSISTANCE AND CAREGIVING
- SECTION M: STRESS
- SECTION O: END OF LIFE PLANNING
- SECTION Q: PSYCHOSOCIAL

We have decided to analyze the features by sections...

In [1]:
# Import libraries
from sas7bdat import SAS7BDAT

import pandas as pd
import numpy as np

from src.section_dict import (
    section_A, # A: DEMOGRAPHICS, IDENTIFIERS, AND WEIGHTS
    section_B, # B: HEALTH
    section_C, # C: HEALTH CARE UTILIZATION AND INSURANCE
    section_D, # D: COGNITION
    section_E, # E: FINANCIAL AND HOUSING WEALTH
    section_F, # F: INCOME
    section_G, # G: FAMILY STRUCTURE
    section_H, # H: EMPLOYMENT HISTORY
    section_I, # I: RETIREMENT
    section_J, # J: PENSION
    section_K, # K: PHYSICAL MEASURES
    section_L, # L: ASSISTANCE AND CAREGIVING
    section_M, # M: STRESS
    section_O, # O: END OF LIFE PLANNING
    section_Q  # Q: PSYCHOSOCIAL
)
# read the dataset from the file (470 MB)
with SAS7BDAT('./dataset/H_MHAS_c2.sas7bdat') as file:
    df = file.to_data_frame()

print('the dataset has '+ str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' features')

[H_MHAS_c2.sas7bdat] column count mismatch


the dataset has 26839 rows and 5241 features


As a first step, we check if all the columns extracted from the guide document are in the dataset...

In [2]:
# Initialize an empty list to store the appended values
Total_sections = []

# Iterate over each dictionary and its values
for section_dict in ( section_A, section_B, section_C, section_D, section_E, section_F, section_G, section_H, section_I, section_J, section_K, section_L, section_M, section_O, section_Q):
    for values_list in section_dict.values():
        # Extend the appended_values list with the values from the current list
        Total_sections.extend(values_list)


# all variables in the dataset...
all_variables = df.columns.tolist()

# Print the appended values
print('The document shows ' + str(len(Total_sections)) +  ' features and the data has ' + str(len(all_variables)) + ' features.\n')

# Searching for variable names that are not in the document...
variables_not_in_list = [variable for variable in all_variables if variable not in Total_sections]

print('The ' + str(len(variables_not_in_list)) + ' variables that are not in the document:')
print(variables_not_in_list)


The document shows 5237 features and the data has 5241 features.

The 4 variables that are not in the document:
['r2relgwk', 's2relgwk', 'r5riccaredpmm', 's1rpfcaredpm']


Since they are not in the PDF guide document, we can delete them.

In [4]:
df.drop(columns=variables_not_in_list,inplace=True)
df.shape

(26839, 5237)

Another approach we can take at this stage is to look for null values; since the data were produced in an interview, many columns contain a high percentage of null data...

In [5]:
# Calculate the percentage of missing data for each column
total_values = df.isnull().sum()

missing_percentage = round((total_values / len(df)) * 100,1)

# Create a DataFrame to store the results
missing_data_df = pd.DataFrame({
    'Total': total_values,
    'Missing Percentage': missing_percentage
})

# Sort the DataFrame by missing percentage in descending order
missing_data_df = missing_data_df.sort_values(by='Missing Percentage', ascending=False)

# Display the top 20 columns with higher missing percentage
top_missing_columns = missing_data_df.head(40)
print("Top 20 columns with higher missing percentage:")
print(top_missing_columns)

Top 20 columns with higher missing percentage:
            Total  Missing Percentage
s5penage    26835               100.0
r3bpref     26839               100.0
s3wghtsft   26827               100.0
s3gripothr  26836               100.0
s2penage    26827               100.0
r5penage    26826               100.0
s5dmonth    26839               100.0
s5dyear     26839               100.0
r3bpsft     26839               100.0
s3bpsft     26839               100.0
s3bpref     26839               100.0
r3wghtref   26837               100.0
s3walksft   26832               100.0
s3walktryu  26832               100.0
s3walkref   26832               100.0
s3walkothr  26832               100.0
r3gripsft   26833               100.0
s3gripsft   26836               100.0
r3gripref   26833               100.0
s3gripref   26836               100.0
s3wghttryu  26827               100.0
r3gripothr  26833               100.0
s3wghtref   26839               100.0
s3hipref    26824                99.9
r3w